Let's use the trap method, simpsons rule, and romberg integration


In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt


# Change function

In [2]:
#define a funtion
def func(x):
    return np.exp( -2*x)*np.cos(10*x)

    

# define core of trap method

In [3]:
def func_integral(x):
    return (np.exp(-2*x)*np.sin(10*x)*5)/52.0  - (np.exp(-2*x)*np.cos(10*x))/52.0

In [4]:
def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h) + f(x)) 

In [5]:
def trapezoid_method(f,a,b,N):
    #f == tunction to integrate
    #a == lower limit if integration
    #b == upper limit of integration
    #N == number of intervals to use
    
    #define x valuesto perform the trapezoid rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    # define the value of the integral
    Fint = 0.0
    
    #perform the integral using the trapezoid method
    for i in range(0,len(x)-1,1):
        Fint += trapezoid_core(f,x[i],h)
        
    #return the answer
    return Fint
    
    

## define core of simpson;s method

In [6]:
def simpsons_core(f,x,h):
    return h*(f(x) + 4*f(x+h) +f(x+2*h))/3.

In [7]:
def simpsons_method(f,a,b,N):
    #f == tunction to integrate
    #a == lower limit if integration
    #b == upper limit of integration
    #N == number of intervals to use
    #note the number of chunks will be N-1
    
    
    #define x valuesto perform the trapezoid rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    # define the value of the integral
    Fint = 0.0
    
    #perform the integral using the 
    #simpsons method

    for i in range(0,len(x)-2,2):
        Fint += simpsons_core(f,x[i],h)
    
    #apply rule over the last interval if N is even
    if((N%2)==2):
        Fint += simpsons_core(f,x[-2],0.5*h)
    #return the answer
    return Fint
    
    

# define romberg core

In [8]:
def romberg_core(f,a,b,i):
    
    #we need the difference between a and b
    h = b-a
    
    #interval between functions evaluations at refine level i
    dh = h/2.**(i)
    
    #we need the cofactor
    K= h/2. **(i+1)
    
    # and the function evaluations
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
        
    #return the answer
    return K*M

## define a wrapper function to perform Romberg integration

In [9]:
def romberg_integration(f,a,b,tol):
    
    #define an iteration variable
    i=0
    
    #define a maximum number of iterations
    imax = 10000
    
    # define an error estimate, set to a large value
    delta = 100.0*np.fabs(tol)
    
    #set an array of integral answers 
    I = np.zeros(imax,dtype=float)
    
    #get the zeroth romberg iteration
    I[0] = 0.5*(b-a)*(f(a) + f(b))
    
    #iterate by 1
    i += 1
    
    while(delta>tol):
        
        #find this romberg iteration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,i)
        
        #compute the new functional error estimate
        delta = np.fabs( (I[i] - I[i-1])/I[i] )
        
        print(i,I[i],I[i-1],delta)
        
        if(delta>tol):
            
            #iterate
            i += 1
            
            #if we've reached the maximum iterations
            if(i>imax):
                print("Max iterations reached.")
                raise StopIteration('Stopping iterations after' ,i)
                
    #return the answer
    return I[i]

In [10]:
integral_value = func_integral(np.pi)-func_integral(0)
print("integral value using actual integration formulae for e^(-2x)cos(10x) = ", integral_value)

# Trapezoid method 
#set initial interval size to 50  
N = 50
tolerance = 1.0e-6
delta = 1 # some high number
while (delta > tolerance):
    N = N + 50
    num_int_value = trapezoid_method(func,0,np.pi,N);
    delta  = np.fabs(num_int_value - integral_value)
print()
print("numeric integral value by Trapezoid method", num_int_value)   
print( "interval size = ",N,"delta = ", delta)

#Simpson method  
N = 50
delta = 1 # some high number
while ((delta) > tolerance):
    N = N + 50
    num_int_value = simpsons_method(func,0,np.pi,N);
    delta  = np.fabs(num_int_value - integral_value)
print()
print("numeric integral value by simpson method", num_int_value) 
print( "interval size = ",N,"delta = ", delta, )
print()
print("Integral value by Romberg method:")
tolerance = 1.0e-6
RI = romberg_integration(func,0,np.pi,tolerance)
print(RI, (RI-integral_value)/integral_value, tolerance)


integral value using actual integration formulae for e^(-2x)cos(10x) =  0.019194856870544078

numeric integral value by Trapezoid method 0.01919582991110986
interval size =  1300 delta =  9.730405657812358e-07

numeric integral value by simpson method 0.019193861813128647
interval size =  5900 delta =  9.950574154302116e-07

Integral value by Romberg method:
1 0.7868648494891817 1.5737296989783633 1.0
2 0.2974797999211515 0.7868648494891817 1.6451034648327187
3 0.1338682162766772 0.2974797999211515 1.222183937270994
4 0.07429884868669352 0.1338682162766772 0.8017535754985687
5 0.04631129469303243 0.07429884868669352 0.6043353825275767
6 0.03265075906247467 0.04631129469303243 0.41838340126854
7 0.02589762452336669 0.03265075906247467 0.26076270173022315
8 0.022539969341304138 0.02589762452336669 0.14896449641170112
9 0.02086584679503983 0.022539969341304138 0.08023266741622372
10 0.02002996035051737 0.02086584679503983 0.041731807247480004
11 0.01961231074592669 0.02002996035051737 0.0

## Number of iterations for Romburg = 26

## Interval size for Trapezoid = 1300

## Interval size for Simpsons method = 5900